In [1]:
"""
Análise de E-commerce Brasileiro - Olist Dataset
Equipe Cedré
Fase 4: KPIs de Negócio e Insights Acionáveis
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

print("="*80)
print("KPIs DE NEGÓCIO E INSIGHTS ACIONÁVEIS")
print("Equipe Cedré")
print("="*80)
print()

# Carregar dados
df = pd.read_csv('data_cleaned.csv')
df_orders = pd.read_csv('orders_aggregated.csv')

# Converter datas
date_cols = ['order_purchase_timestamp', 'order_delivered_customer_date', 
             'order_estimated_delivery_date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col])
    if col in df_orders.columns:
        df_orders[col] = pd.to_datetime(df_orders[col])

df['order_month'] = pd.to_datetime(df['order_month'].astype(str))
df_orders['order_month'] = pd.to_datetime(df_orders['order_month'].astype(str))

print(f"Dataset carregado: {len(df):,} itens, {len(df_orders):,} pedidos")
print()

# ============================================================================
# 1. KPIs PRINCIPAIS
# ============================================================================
print("1. KPIs PRINCIPAIS")
print("="*80)
print()

# 1.1. Receita e Componentes
print("1.1. RECEITA E COMPONENTES")
print("-"*80)

total_revenue = df_orders['total'].sum()
total_subtotal = df_orders['subtotal'].sum()
total_freight = df_orders['freight'].sum()

print(f"Receita Total: R$ {total_revenue:,.2f}")
print(f"  └─ Subtotal (Produtos): R$ {total_subtotal:,.2f} ({total_subtotal/total_revenue*100:.2f}%)")
print(f"  └─ Frete: R$ {total_freight:,.2f} ({total_freight/total_revenue*100:.2f}%)")
print()

# 1.2. Ticket Médio
print("1.2. TICKET MÉDIO")
print("-"*80)

avg_ticket = df_orders['total'].mean()
median_ticket = df_orders['total'].median()

print(f"Ticket Médio: R$ {avg_ticket:.2f}")
print(f"Ticket Mediano: R$ {median_ticket:.2f}")
print()

# 1.3. Frete Médio e Take-rate
print("1.3. FRETE")
print("-"*80)

avg_freight = df_orders['freight'].mean()
freight_share = (total_freight / total_revenue) * 100

print(f"Frete Médio: R$ {avg_freight:.2f}")
print(f"Take-rate de Frete: {freight_share:.2f}%")
print(f"  (Proporção do frete na receita total)")
print()

# 1.4. Desconto
print("1.4. DESCONTO")
print("-"*80)

# Calcular desconto implícito (diferença entre payment_value e total)
df_orders['discount_amount'] = df_orders['payment_value'] - df_orders['total']
df_orders['discount_pct'] = (df_orders['discount_amount'] / df_orders['total'] * 100).clip(-100, 100)

avg_discount_pct = df_orders['discount_pct'].mean()
median_discount_pct = df_orders['discount_pct'].median()

print(f"Desconto Médio: {avg_discount_pct:.2f}%")
print(f"Desconto Mediano: {median_discount_pct:.2f}%")
print(f"⚠ Nota: Valores negativos indicam pagamento > total (possível taxa de serviço)")
print()

# 1.5. Prazo de Entrega
print("1.5. PRAZO DE ENTREGA")
print("-"*80)

avg_delivery_time = df_orders['delivery_lead_time'].mean()
median_delivery_time = df_orders['delivery_lead_time'].median()

print(f"Prazo Médio de Entrega: {avg_delivery_time:.2f} dias")
print(f"Prazo Mediano de Entrega: {median_delivery_time:.2f} dias")
print()

# 1.6. Atraso
print("1.6. ATRASO NA ENTREGA")
print("-"*80)

delivered = df_orders[df_orders['order_status'] == 'delivered']
late_orders = delivered[delivered['is_late'] == 1]

late_rate = (len(late_orders) / len(delivered)) * 100
avg_delay = late_orders['delivery_delay_days'].mean()

print(f"Taxa de Atraso: {late_rate:.2f}%")
print(f"Atraso Médio (quando atrasado): {avg_delay:.2f} dias")
print()

# 1.7. Conversão de Pagamento
print("1.7. CONVERSÃO DE PAGAMENTO")
print("-"*80)

status_counts = df_orders['order_status'].value_counts()
total_orders = len(df_orders)

confirmed_rate = (status_counts.get('delivered', 0) / total_orders) * 100
canceled_rate = (status_counts.get('canceled', 0) / total_orders) * 100

print(f"Taxa de Confirmação (Delivered): {confirmed_rate:.2f}%")
print(f"Taxa de Cancelamento: {canceled_rate:.2f}%")
print()

# Conversão por tipo de pagamento
print("Conversão por Tipo de Pagamento:")
payment_conversion = df_orders.groupby('payment_type').agg({
    'order_id': 'count',
    'is_confirmed': 'sum',
    'is_canceled': 'sum'
}).reset_index()
payment_conversion['confirmed_rate'] = (payment_conversion['is_confirmed'] / 
                                         payment_conversion['order_id'] * 100)
payment_conversion['canceled_rate'] = (payment_conversion['is_canceled'] / 
                                        payment_conversion['order_id'] * 100)

print(payment_conversion[['payment_type', 'order_id', 'confirmed_rate', 'canceled_rate']].to_string(index=False))
print()

# ============================================================================
# 2. ANÁLISE DE SAZONALIDADE
# ============================================================================
print("2. ANÁLISE DE SAZONALIDADE")
print("="*80)
print()

# 2.1. Sazonalidade Mensal
print("2.1. SAZONALIDADE MENSAL")
print("-"*80)

monthly_kpis = df_orders.groupby('order_month').agg({
    'order_id': 'count',
    'total': 'sum',
    'freight': 'mean',
    'delivery_lead_time': 'mean'
}).reset_index()
monthly_kpis.columns = ['month', 'n_orders', 'revenue', 'avg_freight', 'avg_delivery_time']

# Identificar mês com maior e menor volume
max_month = monthly_kpis.loc[monthly_kpis['n_orders'].idxmax()]
min_month = monthly_kpis.loc[monthly_kpis['n_orders'].idxmin()]

print(f"Mês com MAIOR volume: {max_month['month'].strftime('%Y-%m')}")
print(f"  └─ {max_month['n_orders']:.0f} pedidos, R$ {max_month['revenue']:,.2f}")
print()
print(f"Mês com MENOR volume: {min_month['month'].strftime('%Y-%m')}")
print(f"  └─ {min_month['n_orders']:.0f} pedidos, R$ {min_month['revenue']:,.2f}")
print()

# 2.2. Sazonalidade por Estado
print("2.2. TOP 5 ESTADOS POR RECEITA")
print("-"*80)

state_kpis = df_orders.groupby('customer_state').agg({
    'order_id': 'count',
    'total': 'sum'
}).reset_index()
state_kpis.columns = ['state', 'n_orders', 'revenue']
state_kpis = state_kpis.sort_values('revenue', ascending=False).head(5)

print(state_kpis.to_string(index=False))
print()

# 2.3. Sazonalidade por Região
print("2.3. ANÁLISE POR REGIÃO")
print("-"*80)

region_kpis = df_orders.groupby('customer_region').agg({
    'order_id': 'count',
    'total': ['sum', 'mean'],
    'freight': 'mean',
    'delivery_lead_time': 'mean',
    'is_late': 'mean'
}).reset_index()
region_kpis.columns = ['region', 'n_orders', 'revenue', 'avg_ticket', 
                        'avg_freight', 'avg_delivery_time', 'late_rate']
region_kpis['late_rate'] = region_kpis['late_rate'] * 100
region_kpis = region_kpis.sort_values('revenue', ascending=False)

print(region_kpis.to_string(index=False))
print()

# ============================================================================
# 3. PERFORMANCE LOGÍSTICA
# ============================================================================
print("3. PERFORMANCE LOGÍSTICA")
print("="*80)
print()

# Análise por região com foco em logística
print("3.1. MÉTRICAS LOGÍSTICAS POR REGIÃO")
print("-"*80)

logistics_by_region = df_orders[df_orders['order_status'] == 'delivered'].groupby('customer_region').agg({
    'delivery_lead_time': ['mean', 'median', 'std'],
    'is_late': 'mean',
    'delivery_delay_days': lambda x: x[x > 0].mean() if (x > 0).any() else 0
}).reset_index()

logistics_by_region.columns = ['region', 'avg_lead_time', 'median_lead_time', 
                                 'std_lead_time', 'late_rate', 'avg_delay_when_late']
logistics_by_region['late_rate'] = logistics_by_region['late_rate'] * 100
logistics_by_region = logistics_by_region.sort_values('late_rate', ascending=False)

print(logistics_by_region.to_string(index=False))
print()

print("INSIGHTS LOGÍSTICOS:")
worst_region = logistics_by_region.iloc[0]
print(f"  • Região com PIOR performance: {worst_region['region']}")
print(f"    - Taxa de atraso: {worst_region['late_rate']:.2f}%")
print(f"    - Prazo médio: {worst_region['avg_lead_time']:.2f} dias")
print()

best_region = logistics_by_region.iloc[-1]
print(f"  • Região com MELHOR performance: {best_region['region']}")
print(f"    - Taxa de atraso: {best_region['late_rate']:.2f}%")
print(f"    - Prazo médio: {best_region['avg_lead_time']:.2f} dias")
print()

# ============================================================================
# 4. MIX DE PRODUTOS
# ============================================================================
print("4. MIX DE PRODUTOS")
print("="*80)
print()

print("4.1. TOP 10 CATEGORIAS POR RECEITA")
print("-"*80)

category_kpis = df.groupby('product_category_name_english').agg({
    'order_id': 'count',
    'total': 'sum',
    'price': 'mean'
}).reset_index()
category_kpis.columns = ['category', 'n_items', 'revenue', 'avg_price']
category_kpis = category_kpis.sort_values('revenue', ascending=False).head(10)

print(category_kpis.to_string(index=False))
print()

# ============================================================================
# 5. INSIGHTS ACIONÁVEIS
# ============================================================================
print("5. INSIGHTS ACIONÁVEIS")
print("="*80)
print()

insights = []

# Insight 1: Ticket Médio por Forma de Pagamento
credit_ticket = df_orders[df_orders['payment_type'] == 'credit_card']['total'].mean()
boleto_ticket = df_orders[df_orders['payment_type'] == 'boleto']['total'].mean()
diff_pct = ((credit_ticket - boleto_ticket) / boleto_ticket) * 100

insights.append({
    'id': 1,
    'categoria': 'Receita',
    'insight': f'Clientes que pagam com cartão de crédito têm ticket médio {diff_pct:.1f}% maior (R$ {credit_ticket:.2f}) que boleto (R$ {boleto_ticket:.2f})',
    'acao': 'Incentivar pagamento com cartão através de cashback ou parcelamento sem juros'
})

# Insight 2: Taxa de Atraso Regional
norte_late = logistics_by_region[logistics_by_region['region'] == 'Norte']['late_rate'].values[0]
sudeste_late = logistics_by_region[logistics_by_region['region'] == 'Sudeste']['late_rate'].values[0]

insights.append({
    'id': 2,
    'categoria': 'Logística',
    'insight': f'Região Norte tem taxa de atraso {norte_late:.1f}% vs {sudeste_late:.1f}% no Sudeste - diferença de {norte_late-sudeste_late:.1f} pontos percentuais',
    'acao': 'Revisar parceiros logísticos no Norte e ajustar prazos estimados para reduzir insatisfação'
})

# Insight 3: Frete como % da Receita
insights.append({
    'id': 3,
    'categoria': 'Frete',
    'insight': f'Frete representa {freight_share:.2f}% da receita total - valor significativo que impacta margem',
    'acao': 'Negociar melhores tarifas com transportadoras e considerar frete grátis acima de valor mínimo'
})

# Insight 4: Sazonalidade
growth = ((monthly_kpis['n_orders'].iloc[-3:].mean() - 
           monthly_kpis['n_orders'].iloc[:3].mean()) / 
          monthly_kpis['n_orders'].iloc[:3].mean() * 100)

insights.append({
    'id': 4,
    'categoria': 'Sazonalidade',
    'insight': f'Crescimento de {growth:.1f}% no volume de pedidos comparando início e fim do período',
    'acao': 'Preparar estoque e capacidade logística para períodos de pico (Black Friday, Natal)'
})

# Insight 5: Taxa de Cancelamento
insights.append({
    'id': 5,
    'categoria': 'Conversão',
    'insight': f'Taxa de cancelamento de {canceled_rate:.2f}% é baixa, mas representa {status_counts.get("canceled", 0)} pedidos perdidos',
    'acao': 'Investigar motivos de cancelamento e implementar recuperação de carrinho abandonado'
})

# Imprimir insights
for insight in insights:
    print(f"INSIGHT #{insight['id']}: {insight['categoria'].upper()}")
    print(f"  📊 {insight['insight']}")
    print(f"  🎯 Ação: {insight['acao']}")
    print()

# ============================================================================
# 6. DASHBOARD DE KPIs
# ============================================================================
print("6. GERANDO DASHBOARD DE KPIs")
print("-"*80)

# Criar visualização consolidada de KPIs
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(4, 3, hspace=0.4, wspace=0.3)

# KPI 1: Receita Total
ax1 = fig.add_subplot(gs[0, 0])
ax1.text(0.5, 0.6, f'R$ {total_revenue/1e6:.2f}M', 
         ha='center', va='center', fontsize=24, fontweight='bold', color='darkgreen')
ax1.text(0.5, 0.3, 'Receita Total', ha='center', va='center', fontsize=12)
ax1.axis('off')
ax1.set_facecolor('#e8f5e9')

# KPI 2: Ticket Médio
ax2 = fig.add_subplot(gs[0, 1])
ax2.text(0.5, 0.6, f'R$ {avg_ticket:.2f}', 
         ha='center', va='center', fontsize=24, fontweight='bold', color='steelblue')
ax2.text(0.5, 0.3, 'Ticket Médio', ha='center', va='center', fontsize=12)
ax2.axis('off')
ax2.set_facecolor('#e3f2fd')

# KPI 3: Taxa de Confirmação
ax3 = fig.add_subplot(gs[0, 2])
ax3.text(0.5, 0.6, f'{confirmed_rate:.2f}%', 
         ha='center', va='center', fontsize=24, fontweight='bold', color='darkorange')
ax3.text(0.5, 0.3, 'Taxa de Confirmação', ha='center', va='center', fontsize=12)
ax3.axis('off')
ax3.set_facecolor('#fff3e0')

# KPI 4: Prazo Médio
ax4 = fig.add_subplot(gs[1, 0])
ax4.text(0.5, 0.6, f'{avg_delivery_time:.1f} dias', 
         ha='center', va='center', fontsize=24, fontweight='bold', color='purple')
ax4.text(0.5, 0.3, 'Prazo Médio de Entrega', ha='center', va='center', fontsize=12)
ax4.axis('off')
ax4.set_facecolor('#f3e5f5')

# KPI 5: Taxa de Atraso
ax5 = fig.add_subplot(gs[1, 1])
ax5.text(0.5, 0.6, f'{late_rate:.2f}%', 
         ha='center', va='center', fontsize=24, fontweight='bold', color='crimson')
ax5.text(0.5, 0.3, 'Taxa de Atraso', ha='center', va='center', fontsize=12)
ax5.axis('off')
ax5.set_facecolor('#ffebee')

# KPI 6: Take-rate Frete
ax6 = fig.add_subplot(gs[1, 2])
ax6.text(0.5, 0.6, f'{freight_share:.2f}%', 
         ha='center', va='center', fontsize=24, fontweight='bold', color='teal')
ax6.text(0.5, 0.3, 'Take-rate de Frete', ha='center', va='center', fontsize=12)
ax6.axis('off')
ax6.set_facecolor('#e0f2f1')

# Gráfico 7: Evolução Mensal de Receita
ax7 = fig.add_subplot(gs[2, :])
ax7.plot(monthly_kpis['month'], monthly_kpis['revenue']/1e3, 
         marker='o', linewidth=2, markersize=6, color='darkgreen')
ax7.set_xlabel('Mês')
ax7.set_ylabel('Receita (R$ mil)')
ax7.set_title('Evolução Mensal da Receita', fontweight='bold')
ax7.grid(True, alpha=0.3)
ax7.tick_params(axis='x', rotation=45)

# Gráfico 8: Receita por Região
ax8 = fig.add_subplot(gs[3, :2])
ax8.barh(region_kpis['region'], region_kpis['revenue']/1e3, color='steelblue')
ax8.set_xlabel('Receita (R$ mil)')
ax8.set_title('Receita por Região', fontweight='bold')
ax8.invert_yaxis()
ax8.grid(True, alpha=0.3, axis='x')

# Gráfico 9: Taxa de Atraso por Região
ax9 = fig.add_subplot(gs[3, 2])
colors = ['salmon' if x > 7 else 'lightgreen' for x in logistics_by_region['late_rate']]
ax9.barh(logistics_by_region['region'], logistics_by_region['late_rate'], color=colors)
ax9.set_xlabel('Taxa de Atraso (%)')
ax9.set_title('Taxa de Atraso por Região', fontweight='bold')
ax9.invert_yaxis()
ax9.grid(True, alpha=0.3, axis='x')

plt.suptitle('Dashboard de KPIs - E-commerce Olist', fontsize=16, fontweight='bold', y=0.995)
plt.savefig('./outputs/04_kpis_insights/kpis_dashboard.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Salvo: kpis_dashboard.png")

# ============================================================================
# 7. SALVAR RESUMO DE KPIs
# ============================================================================
print("\n7. SALVANDO RESUMO DE KPIs")
print("-"*80)

kpi_summary = pd.DataFrame({
    'KPI': [
        'Receita Total',
        'Ticket Médio',
        'Frete Médio',
        'Take-rate de Frete',
        'Prazo Médio de Entrega',
        'Taxa de Atraso',
        'Taxa de Confirmação',
        'Taxa de Cancelamento'
    ],
    'Valor': [
        f'R$ {total_revenue:,.2f}',
        f'R$ {avg_ticket:.2f}',
        f'R$ {avg_freight:.2f}',
        f'{freight_share:.2f}%',
        f'{avg_delivery_time:.2f} dias',
        f'{late_rate:.2f}%',
        f'{confirmed_rate:.2f}%',
        f'{canceled_rate:.2f}%'
    ]
})

kpi_summary.to_csv('./outputs/04_kpis_insights/kpi_summary.csv', index=False)
print("✓ Arquivo salvo: kpi_summary.csv")

# Salvar insights
insights_df = pd.DataFrame(insights)
insights_df.to_csv('./outputs/04_kpis_insights/insights_acionaveis.csv', index=False)
print("✓ Arquivo salvo: insights_acionaveis.csv")

print()
print("="*80)
print("✓ ANÁLISE DE KPIs E INSIGHTS CONCLUÍDA COM SUCESSO!")
print("="*80)


KPIs DE NEGÓCIO E INSIGHTS ACIONÁVEIS
Equipe Cedré

Dataset carregado: 112,640 itens, 98,658 pedidos

1. KPIs PRINCIPAIS

1.1. RECEITA E COMPONENTES
--------------------------------------------------------------------------------
Receita Total: R$ 15,841,171.67
  └─ Subtotal (Produtos): R$ 13,589,426.41 (85.79%)
  └─ Frete: R$ 2,251,745.26 (14.21%)

1.2. TICKET MÉDIO
--------------------------------------------------------------------------------
Ticket Médio: R$ 160.57
Ticket Mediano: R$ 105.29

1.3. FRETE
--------------------------------------------------------------------------------
Frete Médio: R$ 22.82
Take-rate de Frete: 14.21%
  (Proporção do frete na receita total)

1.4. DESCONTO
--------------------------------------------------------------------------------
Desconto Médio: 0.02%
Desconto Mediano: 0.00%
⚠ Nota: Valores negativos indicam pagamento > total (possível taxa de serviço)

1.5. PRAZO DE ENTREGA
-------------------------------------------------------------------------